In [1]:
%%javascript
require.config({
   paths: {cytoscape: 'http://localhost:8099/js/cytoscape-2.7.10'}
   })

<IPython.core.display.Javascript object>

In [4]:
import igraph as ig
DEF_SCALING = 100.0
def from_igraph(igraph_network, layout=None, scale=DEF_SCALING):
    new_graph = {}
    network_data = {}
    elements = {}
    nodes = []
    edges = []
    network_attr = igraph_network.attributes() # Convert network attributes
    for key in network_attr:
        network_data[key] = igraph_network[key]
    edges_original = igraph_network.es # get network as a list of edges
    nodes_original = igraph_network.vs
    node_attr = igraph_network.vs.attributes()
    for idx, node in enumerate(nodes_original):
        new_node = {}
        data = {}
        data['id'] = str(node.index)
        data['name'] = str(node.index)
        for key in node_attr:
            data[key] = node[key]
        new_node['data'] = data
        if layout is not None:
            position = {}
            position['x'] = layout[idx][0] * scale
            position['y'] = layout[idx][1] * scale
            new_node['position'] = position
        nodes.append(new_node)
    edge_attr = igraph_network.es.attributes()  # Add edges to the elements
    for edge in edges_original:
        new_edge = {}
        data = {}
        data['source'] = str(edge.source)
        data['target'] = str(edge.target)
        for key in edge_attr:
            data[key] = edge[key]
        new_edge['data'] = data
        edges.append(new_edge)
    elements['nodes'] = nodes
    elements['edges'] = edges
    new_graph['elements'] = elements
    new_graph['data'] = network_data
    return new_graph


def to_igraph(network):

    nodes = network['elements']['nodes']
    edges = network['elements']['edges']
    network_attr = network['data']

    node_count = len(nodes)
    edge_count = len(edges)

    g = ig.Graph()

    # Graph attributes
    for key in network_attr.keys():
        g[key] = network_attr[key]

    g.add_vertices(nodes)

    # Add node attributes
    node_attributes = {}
    node_id_dict = {}
    for i, node in enumerate(nodes):
        data = node['data']
        for key in data.keys():
            if key not in node_attributes:
                node_attributes[key] = [None] * node_count

            # Save index to map
            if key == 'id':
                node_id_dict[data[key]] = i

            node_attributes[key][i] = data[key]

    for key in node_attributes.keys():
        g.vs[key] = node_attributes[key]

    # Create edges
    edge_tuples = []
    edge_attributes = {}
    for i, edge in enumerate(edges):
        data = edge['data']
        source = data['source']
        target = data['target']
        edge_tuple = (node_id_dict[source], node_id_dict[target])
        edge_tuples.append(edge_tuple)
        for key in data.keys():
            if key not in edge_attributes:
                edge_attributes[key] = [None] * edge_count

            # Save index to map
            edge_attributes[key][i] = data[key]

    g.add_edges(edge_tuples)

    # Assign edge attributes
    for key in edge_attributes.keys():
        if key == 'source' or key == 'target':
            continue
        else:
            g.es[key] = edge_attributes[key]

    return g


In [6]:
from igraph import *
lines = open("test.tsv").readlines()
abRelations = []
for line in lines:
    abRelations.append(line.split("\t"))
g = Graph.TupleList(abRelations)
g.json = from_igraph(g)

In [7]:
g.json

{'data': {},
 'elements': {'edges': [{'data': {'source': '0', 'target': '1'}},
   {'data': {'source': '1', 'target': '2'}},
   {'data': {'source': '3', 'target': '4'}},
   {'data': {'source': '3', 'target': '5'}},
   {'data': {'source': '5', 'target': '6'}},
   {'data': {'source': '5', 'target': '7'}},
   {'data': {'source': '5', 'target': '8'}}],
  'nodes': [{'data': {'id': '0', 'name': 'M_10fthf6glu_m'}},
   {'data': {'id': '1', 'name': 'R_FPGS9m\n'}},
   {'data': {'id': '2', 'name': 'M_glu_DASH_DASH_DASH_L_m'}},
   {'data': {'id': '3', 'name': 'R_FPGS9m'}},
   {'data': {'id': '4', 'name': 'M_10fthf7glu_m\n'}},
   {'data': {'id': '5', 'name': 'M_pi_m\n'}},
   {'data': {'id': '6', 'name': 'R_PPCOACm'}},
   {'data': {'id': '7', 'name': 'R_SUCOAS1m'}},
   {'data': {'id': '8', 'name': 'R_SUCOASm'}}]}}

In [9]:
import ipywidgets as widgets
import json
import time
from IPython.display import display
from traitlets import Int, Unicode, Tuple, CInt, Dict, validate, observe

class cyjsWidget(widgets.DOMWidget):
    
    _view_name = Unicode('CyjsView').tag(sync=True)
    _view_module = Unicode('cyjs').tag(sync=True)
    frameWidth = Int(400).tag(sync=True)
    frameHeight = Int(300).tag(sync=True)
    msgFromKernel = Unicode("{}").tag(sync=True)
    msgFromBrowser = Unicode("{}").tag(sync=True)
    _incomingMessage = {};
    status = "initial status message\n"
    selectedNodes = [];
        
    def setSize(self, width, height):
      self.status += "setSize(%d, %d)\n" % (width, height)
      self.frameWidth = width
      self.frameHeight = height
        
    def fit(self, margin=50):
      self.status += "entering fit (%d)\n" % margin
      self.msgFromKernel = json.dumps({"cmd": "fit", "status": "request",
                                              "callback": "", "payload": margin});
        
    def getSelectedNodes(self):
      #self.selectedNodes = [];
      self._incomingMessage = None;
      self.status += "entering getSelectedNodes\n";
      self.msgFromKernel = json.dumps({"cmd": "cleanSlate", "status": "request", "callback": "", "payload":""});
      self.msgFromKernel = json.dumps({"cmd": "getSelectedNodes", "status": "request",
                                       "callback": "", "payload": ""});
    
    def selectNodes(self, nodes):
      self.msgFromKernel = json.dumps({"cmd": "selectNodes", "status": "request",
                                       "callback": "", "payload": nodes});
       
    def clearSelection(self):
      self. msgFromKernel = json.dumps({"cmd": "clearSelection", "status": "request",
                                        "callback": "", "payload": ""});
        
    @observe('msgFromBrowser')
    def msg_arrived(self, change):
       self.status += "msgFromBrowser has arrived: %f\n" % time.time()
       rawMessage = change['new']
       self._incomingMessage = json.loads(rawMessage)
         
    def getResponse(self):
       return(self._incomingMessage["payload"])

    def getFullResponse(self):
       return(self._incomingMessage)
        

In [10]:
%%javascript
"use strict";

require.undef('cyjs');

define('cyjs', ["jupyter-js-widgets", "cytoscape"], function(widgets, cytoscape) {
    
    var CyjsView = widgets.DOMWidgetView.extend({

        initialize: function() {
           this.circles = [];
           this.circleCount = 0;
           this.options = {}; 
           this.msg = "empty in javascript";
           this.msgFromKernel = "";
           this.defaultHeight = "800px";
           this.defaultWidth = "1000px";
           },

        createDiv: function(){
            var outerDiv = $("<div id='cyOuterDiv' style='border:1px solid gray; height: 800px; width: 1000px'></div>");
            var toolbarDiv = $("<div id='cyToolbarDiv' style='height: 30px; width: 1000px'></div>");
            var cyDiv = $("<div id='cyDiv' style='height: 870px; width: 1000px'></div>");
            outerDiv.append(toolbarDiv);
            outerDiv.append(cyDiv);
            var cyWidget = this;
            var fitButton = $("<button>Fit</button>").click(function(){
                console.log("Fit!");
                console.log("fitButton's notion of this:")
                console.log(cyWidget.cy);
                cyWidget.cy.fit(50);
               });
            toolbarDiv.append(fitButton);
            var fitSelectedButton = $("<button>Fit Selected</button>").click(function(){
                var selectedNodes = cyWidget.cy.filter('node:selected');
                if(selectedNodes.length > 0){
                   cyWidget.cy.fit(selectedNodes, 50);
                   }
               });
            toolbarDiv.append(fitSelectedButton);
            var sfnButton = $("<button>SFN</button>").click(function(){
               cyWidget.cy.nodes(':selected').neighborhood().nodes().select()
               });
            toolbarDiv.append(sfnButton);
            var clearButton = $("<button>Clear</button>").click(function(){
               cyWidget.cy.nodes().unselect();
               cyWidget.cy.edges().unselect();
               });
            toolbarDiv.append(clearButton);
            return(outerDiv);
           },
 
        
        createCanvas: function(){
            var cyjsWidget = this;
            console.log("createCanvas notion of this:")
            console.log(cyjsWidget);
            this.cy = cytoscape({
               container: document.getElementById('cyDiv'),
               //elements: {
               //nodes: [
               //  {data: {id: 'a', name: 'Node A', type: 'big' }},
               //  {data: {id: 'b', name: 'Node B', type: 'little'}},
               //  ],
              //edges: [
              //  {data: {source: 'a', target: 'b'}},
              //         {data: {source: 'b', target: 'a'}}
              // ]},
          ready: function(){
            console.log("small cyjs network ready");
            console.log("ready's notion of this:")
            console.log(this);
            cyjsWidget.cy = this;
            window.cy = this;  // for easy debugging
            console.log("ready's notion of cyjsWidget:")
            console.log(cyjsWidget);
            console.log("calling this.fit")
            //cyWidget.cy.fit(100);
            console.log("--- about to call loadGraph")
            cyjsWidget.loadGraph("priyanka.json");
            cyjsWidget.loadStyle("style.js");
            console.log("    back from loadGraph")
            cyjsWidget.cy.on("select", function(x){
                var selectedNodeCount = cyjsWidget.cy.nodes(":selected").length;
                var selectedEdgeCount = cyjsWidget.cy.edges(":selected").length;
                console.log("selected nodes: " + selectedNodeCount);
                console.log("selected edges:" + selectedEdgeCount);
                });
            cyjsWidget.cy.on("unselect", function(x){
                var selectedNodeCount = cyjsWidget.cy.nodes(":selected").length;
                var selectedEdgeCount = cyjsWidget.cy.edges(":selected").length;
                console.log("selected nodes: " + selectedNodeCount);
                console.log("selected edges:" + selectedEdgeCount);
                });
            } // ready
           })},

        loadStyle: function(filename){
           var cyObj = this.cy;
           console.log("cyjsWidget.loadStyle: " + filename)
           console.log("loadStyle's notion of this:");
           console.log(this);
           console.log("loadStyle's notion of cy:");
           console.log(cyObj);
           var str = window.location.href;
           var url = str.substr(0, str.lastIndexOf("/")) + "/" + filename;
           url = url.replace("/notebooks/", "/files/");
           console.log("about to getScript: " + url);
           $.getScript(url)
              .done(function(script, textStatus) {
                 console.log(textStatus);
                 cyObj.style(vizmap);
                 })
             .fail(function( jqxhr, settings, exception ) {
                console.log("getScript error trying to read " + filename);
                console.log("exception: ");
                console.log(exception);
                });
          },
        
        loadGraph: function(filename){
           console.log("entering loadGraph");
           var cyObj = this.cy;
              // the robust url of a file in the same directory as the notebook is
              // str.substring(0, str.lastIndexOf("/"));
           var str = window.location.href;
           var url = str.substr(0, str.lastIndexOf("/")) + "/" + filename;
           url = url.replace("/notebooks/", "/files/");
           console.log("=== about to getScript on " + url);
           $.getScript(url)
              .done(function(script, textStatus) {
                 console.log("getScript: " + textStatus);
                 console.log("nodes: " + network.elements.nodes.length);
                 if(typeof(network.elements.edges) != "undefined")
                    console.log("edges: " + network.elements.edges.length);
                cyObj.add(network.elements);  // no positions yet
                cyObj.nodes().map(function(node){node.data({degree: node.degree()})});
                cyObj.fit(150);
                }) // .done
            .fail(function(jqxhr, settings, exception) {
               console.log("addNetwork getscript error trying to read " + filename);
               });
           },
        
        render: function() { 
            console.log("entering render")
            this.$el.append(this.createDiv());
            this.listenTo(this.model, 'change:frameWidth', this.frameDimensionsChanged, this);
            this.listenTo(this.model, 'change:frameHeight', this.frameDimensionsChanged, this);
            this.listenTo(this.model, 'change:msgFromKernel', this.dispatchRequest, this);
            var cyjsWidget = this;
            function myFunc(){
               cyjsWidget.createCanvas()
               }
            setTimeout(myFunc, 500);
            },

        dispatchRequest: function(){
           console.log("dispatchRequest");
           var msgRaw = this.model.get("msgFromKernel");
           var msg = JSON.parse(msgRaw);
           console.log(msg);
           console.log("========================");
           console.log(this);
           switch(msg.cmd) {
              case 'fit':
                 var margin = msg.payload;
                 console.log("fit with margin: " + margin)
                 this.cy.fit(margin);
                 break;
              case 'getSelectedNodes':
                 var selectedNodes = this.cy.filter("node:selected").map(function(node){ 
                     return node.data().id});
                  console.log("-- found these selected nodes: ");
                  console.log(selectedNodes);
                  var jsonString = JSON.stringify({cmd: "storeSelectedNodes",
                                                status: "reply",
                                                callback: "",
                                                payload: selectedNodes})
                  console.log(" *** jsonString: ")
                  console.log(jsonString);
                  this.model.set("msgFromBrowser", jsonString);
                  console.log("    after setting 'msgFromBrowser");
                  this.touch();
                  break;
               case 'selectNodes':
                  var nodeIDs = msg.payload;
                  console.log("--- selecting these nodes: " + nodeIDs);
                  if(typeof(nodeIDs) == "string")
                     nodeIDs = [nodeIDs];
                 var filterStrings = [];
                 for(var i=0; i < nodeIDs.length; i++){
                   var s = '[id="' + nodeIDs[i] + '"]';
                   filterStrings.push(s);
                   } // for i
                var nodesToSelect = this.cy.nodes(filterStrings.join());
                nodesToSelect.select()
                break;
              case 'clearSelection':
                 this.cy.nodes().unselect();
                 break;
            default:
               console.log("unrecognized msg.cmd: " + msg.cmd);
             } // switch
           console.log("CONCLUDING dispatchRequest")
           }, 
        
        frameDimensionsChanged: function(){
           console.log("frameDimensionsChanged")
           var newWidth  = this.model.get("frameWidth");
           var newHeight = this.model.get("frameHeight");
           console.log("frame: " + newWidth + " x " + newHeight);
           $("#cyOuterDiv").width(newWidth);
           $("#cyOuterDiv").height(newHeight);
           $("#cyToolbarDiv").width(newWidth);
           $("#cyDiv").width(newWidth);
           $("#cyDiv").height(newHeight - $("#cyToolbarDiv").height());
           }, 
        
        events: {
           //"click #svg": "changeHandler"
           }

    });
    return {
        CyjsView: CyjsView
    };
});

<IPython.core.display.Javascript object>

In [11]:
cy = cyjsWidget()
#cy

In [12]:
display(cy)

In [13]:
cy.clearSelection()

In [14]:
print(cy.getSelectedNodes())

None


In [15]:
print(cy.status)

initial status message
entering getSelectedNodes
msgFromBrowser has arrived: 1480541340.915815



In [232]:
time.sleep(1)

In [233]:
# cy.getResponse()

In [234]:
# cy.getFullResponse()

In [235]:
cy.selectNodes(["MAG", "PLD2"])

In [236]:
# cy.getSelectedNodes()

In [237]:
# cy.getResponse()

In [28]:
import IPython, zmq
ipython = IPython.get_ipython()
hasattr(ipython, 'kernel')
ioloop = zmq.eventloop.ioloop.IOLoop.instance()

In [29]:
ioloop

In [34]:
def work():
    for i in range(10):
        def update_progress(i=i):
            print("calling from thread", threading.currentThread())
        print(i)
        time.sleep(0.5)
        ioloop.add_callback(update_progress)


In [35]:
import threading
thread = threading.Thread(target=work)

In [36]:
thread.start()

0
1
calling from thread <_MainThread(MainThread, started 140735095894016)>
2calling from thread <_MainThread(MainThread, started 140735095894016)>

3
calling from thread <_MainThread(MainThread, started 140735095894016)>
4
calling from thread <_MainThread(MainThread, started 140735095894016)>
5
calling from thread <_MainThread(MainThread, started 140735095894016)>
6
calling from thread <_MainThread(MainThread, started 140735095894016)>
7
calling from thread <_MainThread(MainThread, started 140735095894016)>
8
calling from thread <_MainThread(MainThread, started 140735095894016)>
9
calling from thread <_MainThread(MainThread, started 140735095894016)>
calling from thread <_MainThread(MainThread, started 140735095894016)>


In [37]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %install_default_config  %install_ext  %install_profiles  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%latex  %%

In [39]:
%system ls


['network.json',
 'out_file.html',
 'simple.ipynb',
 'style.js',
 'waitOnBrowserResponse.ipynb']

In [ ]:
%debug

> <ipython-input-31-4be24dfd3ef9>(1)<module>()
----> 1 thread = threading.Thread(target=work)

ipdb> 2 + 2
4
